In [17]:
import pandas as pd

data_path = "../data/"

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path+'test.csv')
submission = pd.read_csv(data_path+ 'sampleSubmission.csv')


In [18]:
train = train[train['weather'] != 4]

all_data = pd.concat([train,test], ignore_index=True)

In [19]:
all_data['datetime'] = pd.to_datetime(all_data['datetime'])

all_data['year'] = all_data['datetime'].dt.year
all_data['month'] = all_data['datetime'].dt.month
all_data['day'] = all_data['datetime'].dt.day
all_data['hour'] = all_data['datetime'].dt.hour
all_data['minute'] = all_data['datetime'].dt.minute
all_data['second'] = all_data['datetime'].dt.second
all_data['weekday'] = all_data['datetime'].dt.weekday
all_data.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour,minute,second,weekday
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3.0,13.0,16.0,2011,1,1,0,0,0,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8.0,32.0,40.0,2011,1,1,1,0,0,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5.0,27.0,32.0,2011,1,1,2,0,0,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3.0,10.0,13.0,2011,1,1,3,0,0,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0.0,1.0,1.0,2011,1,1,4,0,0,5


In [22]:
drop_features = ['casual', 'registered', 'datetime','month','windspeed']
all_data = all_data.drop(drop_features, axis =1)

In [26]:
X_train = all_data[~pd.isnull(all_data['count'])].drop(['count'],axis=1)
X_test = all_data[pd.isnull(all_data['count'])].drop(['count'],axis=1)

y=train['count']

In [27]:
import numpy as np

def rmsle(y_true, y_pred, convertExp=True):
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)

    log_true = np.nan_to_num(np.log(y_true+1))
    log_pred = np.nan_to_num(np.log(y_pred+1))

    output = np.sqrt(np.mean(log_true - log_pred)**2)
    return output


In [28]:
from sklearn.linear_model import LinearRegression

linear_reg_model = LinearRegression()

log_y = np.log(y)
linear_reg_model.fit(X_train, log_y)


LinearRegression()

In [32]:
preds = linear_reg_model.predict(X_train)
f'RMSLE값 : {rmsle(log_y, preds, True):.4f}'

'RMSLE값 : 0.0215'

In [36]:
linearreg_preds = linear_reg_model.predict(X_test)
submission['count'] = np.exp(linearreg_preds)
submission.to_csv('submission.csv', index= False)